In [1]:
from training_functions import train_model
from data_helper_functions import get_tournament_ids_by_year

# Get all games from tournaments in 2023
tournament_ids = get_tournament_ids_by_year(2023)

/Users/sam/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# load all necessary data to csv

import pandas as pd
from training_functions import load_data_to_csv
load_data_to_csv(tournament_ids)

Removing data from previous tournament...

Data removed successfully.

Writing tournament #2 Summer 2023 game files...

----- Processed 10 games, current run time:                                 0.26 minutes
ESPORTSTMNT03:3196057 110733838936446954 not found in the mapping table
----- Processed 20 games, current run time:                                 0.69 minutes
----- Processed 30 games, current run time:                                 1.07 minutes
ESPORTSTMNT03:3198296 110733838936447001 not found in the mapping table
ESPORTSTMNT03:3198296 110733838936447002 not found in the mapping table
----- Processed 40 games, current run time:                                 1.5 minutes
----- Processed 50 games, current run time:                                 1.77 minutes
----- Processed 60 games, current run time:                                 2.16 minutes
----- Processed 61 games total, in: 2.18 minutes
Games loaded: 10
Games loaded: 20
Games loaded: 30
Games loaded: 40
Total Games Fo

In [24]:
import numpy as np
df = pd.read_csv("data/df_csv/features_final.csv", index_col=[0])

In [25]:
# Rewrite dataframe, results were reversed in processing
df['RESULT']  = np.where(df['RESULT'] == 0, 1, 0)

In [26]:
df

,level,shutdownValue,totalGold,MINIONS_KILLED,NEUTRAL_MINIONS_KILLED,NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE,NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE,CHAMPIONS_KILLED,NUM_DEATHS,ASSISTS,...,TOTAL_DAMAGE_DEALT_TO_OBJECTIVES,TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS,TOTAL_HEAL_ON_TEAMMATES,participantID,CHAMPION,SUMMONER_NAME,ESPORTS_ID,PLATFORM_ID,SIDE,RESULT
0,15,150,10450,178,6.000000,0.000000,6.000000,2,0,10,...,5182.645508,212.971283,0.000000,1,Poppy,MIR Allorim,110733838936446929,ESPORTSTMNT03:3196037,100,1
1,14,400,11406,17,114.400017,67.500008,22.399998,5,2,10,...,33652.128906,73.234192,0.000000,2,Rengar,MIR Chad,110733838936446929,ESPORTSTMNT03:3196037,100,1
2,14,300,10481,191,0.000000,0.000000,0.000000,4,4,8,...,8488.514648,0.766968,0.000000,3,Kaisa,MIR Sudzzi,110733838936446929,ESPORTSTMNT03:3196037,100,1
3,13,700,13612,204,43.600006,19.999996,10.100000,10,1,6,...,36039.250000,39.625183,0.000000,4,Xayah,MIR Neo,110733838936446929,ESPORTSTMNT03:3196037,100,1
4,13,0,7706,31,0.000000,0.000000,0.000000,1,0,16,...,2568.289307,43.130798,1439.260132,5,Zilean,MIR Dardoch,110733838936446929,ESPORTSTMNT03:3196037,100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,16,300,11631,213,0.000000,0.000000,0.000000,3,0,10,...,13039.763672,13.730011,1249.557007,1,Illaoi,GK Satorius,109545777184190004,ESPORTSTMNT01:3337315,100,1
696,14,150,10146,10,119.150024,73.400017,8.500000,2,0,14,...,51663.648438,121.539322,1845.673462,2,Maokai,GK Viking,109545777184190004,ESPORTSTMNT01:3337315,100,1
697,16,150,12621,213,8.750000,7.000000,0.000000,5,4,12,...,14851.511719,91.283295,0.000000,3,Cassiopeia,GK Giyuu,109545777184190004,ESPORTSTMNT01:3337315,100,1
698,15,700,15902,277,12.000000,8.000000,0.000000,10,2,7,...,28625.328125,66.547600,760.497253,4,Varus,GK Rin,109545777184190004,ESPORTSTMNT01:3337315,100,1


In [27]:
from sklearn.model_selection import train_test_split
from weighter import Weighter

column_names = df.columns

metadata_columns = ["RESULT", "CHAMPION", "participantID", "SUMMONER_NAME", "ESPORTS_ID", "PLATFORM_ID"]

weighter = Weighter()

#splt data into test and train
X = df.drop(columns=metadata_columns, axis=1).values
y = df["RESULT"].values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

#fit the model
weighter.fit(column_names=column_names, Xtrain=Xtrain, ytrain=ytrain, scaled=False)
    
accuracy_score = weighter.clf_.score(Xtest, ytest)
print(f"Accuracy Score: {accuracy_score}\n")

Saving weights to JSON.

Accuracy Score: 0.9115319543742837



In [28]:
from pathlib import Path

filepath = Path('data/df_csv/features.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 